In [1]:
import PyQt5
%matplotlib qt
import pyzed.sl as sl
from ZED import ZED
from TRACKING import handtracking
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import datetime

In [2]:
LIVE_STREAM = False
svo_file_name = "test.svo"
if LIVE_STREAM == False:
    if svo_file_name is None:
        print('Undefined .svo file name for processing')
        sys.exit()

In [3]:
# Run this Block to Run Hand Tracking with LIVE STREAM VIDEO
if LIVE_STREAM == True:
    zed = ZED(fps=60)
    zed.config_resolution("HD720")
    zed.config_depth_mode("NEURAL")
    zed.open_camera(fill_mode=True)
    zed.get_calibration_params()
    fx = zed.fx
    fy = zed.fy
    cx = zed.cx
    cy = zed.cy
    width = zed.camera.get_camera_information().camera_configuration.resolution.width
    height = zed.camera.get_camera_information().camera_configuration.resolution.height

In [4]:
# Run this Block to Run Hand Tracking with RECORDED VIDEO (.SVO FILE)
# And Specify .svo File Name
if LIVE_STREAM == False:
    input_name = svo_file_name
    zed = ZED(fps=None, input_filename = input_name)
    zed.config_depth_mode("NEURAL")
    zed.open_camera(fill_mode=True)
    zed.get_calibration_params()
    fx = zed.fx
    fy = zed.fy
    cx = zed.cx
    cy = zed.cy
    width = zed.camera.get_camera_information().camera_configuration.resolution.width
    height = zed.camera.get_camera_information().camera_configuration.resolution.height
    svo_frame_rate = zed.camera.get_init_parameters().camera_fps
    num_frames = zed.camera.get_svo_number_of_frames()
    print("The SVO File's Width Is: ", width)
    print("The SVO File's Height Is: ", height)
    print("The SVO File's FPS is: ", svo_frame_rate)
    print("The SVO File's Number of Frames is: ", num_frames)


Reading SVO File: 0
Open ZED Successfully!
The SVO File's Width Is:  1280
The SVO File's Height Is:  720
The SVO File's FPS is:  53
The SVO File's Number of Frames is:  461


In [5]:
detector = handtracking()

In [6]:
fig = plt.figure()
plt.ion()
ax = fig.add_subplot(111, projection='3d')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
while True:
    zed.camera.grab(zed.runtime_params)
    timestamp = zed.camera.get_timestamp(sl.TIME_REFERENCE.IMAGE)
    timestamp = timestamp.get_milliseconds()
    timestamp = datetime.datetime.fromtimestamp(timestamp / 1000)
    print(timestamp)
    zed.get_image()
    image = zed.image_cv2
    point_cloud = zed.point_cloud
    # depth = zed.depth_cv2
    image = detector.detect_landmarks(image)
    data_left, data_right = detector.get_position(image, point_cloud, fx = fx, fy = fy, width=width, height=height)

    if data_right.shape == (21,3) and data_left.shape == (21,3):
        data_plot = np.vstack((data_right,data_left))
    elif data_right.shape == (21,3):
        data_plot = data_right
    elif data_left.shape == (21,3):
        data_plot = data_left
    else:
        data_plot = np.array([])
    detector.plot(ax,plt,data_plot)

    cv2.imshow("test", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
zed.camera.close()

2023-10-17 23:28:39.250000
2023-10-17 23:28:39.267000
2023-10-17 23:28:39.283000
2023-10-17 23:28:39.300000
2023-10-17 23:28:39.317000
2023-10-17 23:28:39.333000
2023-10-17 23:28:39.350000
2023-10-17 23:28:39.367000
2023-10-17 23:28:39.383000
2023-10-17 23:28:39.400000
2023-10-17 23:28:39.417000
2023-10-17 23:28:39.433000
2023-10-17 23:28:39.450000
2023-10-17 23:28:39.467000
2023-10-17 23:28:39.483000
2023-10-17 23:28:39.500000
2023-10-17 23:28:39.517000
2023-10-17 23:28:39.533000
2023-10-17 23:28:39.550000
2023-10-17 23:28:39.567000
2023-10-17 23:28:39.583000
2023-10-17 23:28:39.600000
2023-10-17 23:28:39.617000
2023-10-17 23:28:39.633000
2023-10-17 23:28:39.650000
2023-10-17 23:28:39.667000
2023-10-17 23:28:39.683000
2023-10-17 23:28:39.700000
2023-10-17 23:28:39.717000
2023-10-17 23:28:39.733000
2023-10-17 23:28:39.750000
2023-10-17 23:28:39.767000
2023-10-17 23:28:39.783000
2023-10-17 23:28:39.800000
2023-10-17 23:28:39.817000
2023-10-17 23:28:39.833000
2023-10-17 23:28:39.850000
2

/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:40: RuntimeWarning: invalid value encountered in scalar multiply
  x_3d = wrist_position[0] + (landmark.x * width - wrist_landmark[0] * width) * wrist_position[2]/fx
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:42: RuntimeWarning: invalid value encountered in scalar multiply
  y_3d = wrist_position[1] + (landmark.y * height - wrist_landmark[1] * height) * wrist_position[2]/fy
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:40: RuntimeWarning: invalid value encountered in scalar add
  x_3d = wrist_position[0] + (landmark.x * width - wrist_landmark[0] * width) * wrist_position[2]/fx
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:42: RuntimeWarning: invalid value encountered in scalar add
  y_3d = wrist_position[1] + (landmark.y * height - wrist_landmark[1] * height) * wrist_position[2]/fy


2023-10-17 23:28:40.300000
2023-10-17 23:28:40.317000
2023-10-17 23:28:40.333000
2023-10-17 23:28:40.350000
2023-10-17 23:28:40.367000
2023-10-17 23:28:40.383000
2023-10-17 23:28:40.400000
2023-10-17 23:28:40.417000
2023-10-17 23:28:40.433000
2023-10-17 23:28:40.450000
2023-10-17 23:28:40.467000
2023-10-17 23:28:40.483000
2023-10-17 23:28:40.500000
2023-10-17 23:28:40.517000
2023-10-17 23:28:40.533000
2023-10-17 23:28:40.550000
2023-10-17 23:28:40.567000
2023-10-17 23:28:40.583000
2023-10-17 23:28:40.600000
2023-10-17 23:28:40.617000
2023-10-17 23:28:40.633000
2023-10-17 23:28:40.650000
2023-10-17 23:28:40.667000
2023-10-17 23:28:40.683000
2023-10-17 23:28:40.700000
2023-10-17 23:28:40.717000
2023-10-17 23:28:40.733000
2023-10-17 23:28:40.750000
2023-10-17 23:28:40.767000
2023-10-17 23:28:40.783000
2023-10-17 23:28:40.800000
2023-10-17 23:28:40.817000
2023-10-17 23:28:40.833000
2023-10-17 23:28:40.850000
2023-10-17 23:28:40.867000
2023-10-17 23:28:40.883000
2023-10-17 23:28:40.900000
2